In [1]:
from random import seed,randrange,random
from sklearn.model_selection import train_test_split
import numpy as np
import re
 
# import csv
def loadDataSet(filename):
    dataset = []
    with open(filename, 'r') as fr:
        for line in fr.readlines():
            if not line:
                continue
            lineArr = []
            for feature in line.split(','):
                str_f = feature.strip()
                if str_f[0].isalpha() or 'day' in str_f or re.match("\d{4}-\d{1,2}-\d{1,2}",str_f):
                    if 'Date' not in str_f and not re.match("\d{4}-\d{1,2}-\d{1,2}",str_f):     
                        lineArr.append(str_f)
                else:
                    lineArr.append(float(str_f))
            dataset.append(lineArr)
    return dataset

In [77]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)       
    fold_size = len(dataset) / n_folds
    for i in range(n_folds):
        fold = list()                  
        while len(fold) < fold_size:   
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy[index])  
        dataset_split.append(fold)
    return dataset_split

In [78]:
# Split a dataset based on an attribute and an attribute value 
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [79]:
def gini_index(groups, class_values):    
    gini = 0.0
    D = len(groups[0]) + len(groups[1])
    for class_value in class_values:     # class_values = [0, 1]
        for group in groups:             # groups = (left, right)
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += float(size)/D * (proportion * (1.0 - proportion))    
    return gini

In [80]:
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))  # class_values =[0, 1]
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)  
        if index not in features:
            features.append(index)
    for index in features: 
        for row in dataset:
            groups = test_split(index, row[index], dataset)  
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups  
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

In [81]:
# Create a terminal node value 
def to_terminal(group):
    outcomes = [row[-1] for row in group]           
    return max(set(outcomes), key=outcomes.count)   

In [82]:
# Create child splits for a node or make terminal  
def split(node, max_depth, min_size, n_features, depth):  # max_depth = 10, min_size = 1, n_features=int(sqrt((len(dataset[0])-1)
    left, right = node['groups']
    del(node['groups'])
# check for a no split 
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
# check for max depth
    if depth >= max_depth:   
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
# process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features) 
        split(node['left'], max_depth, min_size, n_features, depth+1)  # recursion
# process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [83]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

In [84]:
# Make a prediction with a decision tree
def predict(node, row):  
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):       
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [85]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [86]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):   
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [87]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
        
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions

In [88]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted): 
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [89]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            row_copy[-1] = None
            test_set.append(row_copy)
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
 
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [74]:
if __name__ == '__main__':
 
    # load data
    dataset = loadDataSet('./pred.csv')

    n_folds = 5        
    max_depth = 20     
    min_size = 1       
    sample_size = 1.0  
    n_features = 15     
    for n_trees in [1,10]:  
        scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
        seed(1)
        print('random=', random())
        print('Trees: %d' % n_trees)
        print('Scores: %s' % scores)
        print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

TypeError: '<' not supported between instances of 'str' and 'float'